In [2]:
from pyowm import OWM
from pyowm.weatherapi25.forecast import Forecast
from pyowm.exceptions.api_response_error import NotFoundError
from pyowm.exceptions.api_call_error import APICallTimeoutError
from config import OWM_API_key as key, connection_port, user, loc_host, remo_host, password, socket_path

API_key = key
owm = OWM(API_key) # the OWM object
print(type(owm))

API_key = 'key'
owm = OWM(API_key) # the OWM object
print(type(owm))

<class 'pyowm.weatherapi25.owm25.OWM25'>
<class 'pyowm.weatherapi25.owm25.OWM25'>


In [5]:
import json
import os
import time
import urllib
import dns

from pyowm import OWM
from pyowm.weatherapi25.forecast import Forecast
from pyowm.exceptions.api_response_error import NotFoundError
from pyowm.exceptions.api_call_error import APICallTimeoutError
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, InvalidDocument, DuplicateKeyError, OperationFailure
from urllib.parse import quote
from config import OWM_API_key as key, connection_port, user, loc_host, remo_host, password, socket_path

global zipcode
global obs
global reception_time
global zid
global zlon
global zlat
global client

API_key = key
loc_host = loc_host
rem_host = remo_host
port = connection_port
owm = OWM(API_key)    # the OWM object
password = quote(password)    # url encode the password for the mongodb uri
uri = "mongodb+srv://%s:%s@%s" % (user, password, socket_path)


def make_zip_list(state):
    ''' Make a list of zip codes in the specified state.
        Read zip codes from downloadable zip codes csv file available at https://www.unitedstateszipcodes.org/
        
        :param state: the two-letter abreviation of the state whose zip codes you'd like listed
        :type state: string
        
        :returns success_zips: list of zip codes that OWM has records for
        :type success_zips: list
    '''
    import pandas as pd
    from pyowm.exceptions.api_response_error import NotFoundError
    
    successes = 0
    exceptions = 0
    success_zips = []
    fail_zips = []
    
    # Make a datafram from the rows with the specified state and write the 'zip' column to a list 
    df = pd.read_csv("resources/zip_code_database.csv")
    zip_list = df.loc[lambda df: df['state'] == f'{state.upper()}']['zip'].tolist()
    for zipp in zip_list:
        if int(zipp) > 10000:
            try:
                set_location(zipp)
                success_zips.append(zipp)
                successes+=1
            except NotFoundError:
                print("except", f'NotFoundError with zipcode {zipp}')
                fail_zips.append(zipp)
                exceptions+=1
                pass
    filename = f'resources/success_zips{state}.csv'
    write_list_to_file(success_zips, filename)
    write_list_to_file(fail_zips, 'resources/fail_zips.csv')
    print(f'successes = {successes}; exceptions = {exceptions}, all written to files')
    return(read_list_from_file(filename))

def write_list_to_file(zip_list, filename):
#     print('using write_list_to_file')
    """ Write the zip codes to csv file.
        
        :param zip_list: the list created from the zip codes dataframe
        :type zip_list: list stings
        :param filename: the name of the file
        :type filename: sting
    """
    with open(filename, "w") as z_list:
        z_list.write(",".join(str(z) for z in zip_list))
    return
        
def read_list_from_file(filename):
    """ Read the zip codes list from the csv file.
        
        :param filename: the name of the file
        :type filename: sting
    """
    with open(filename, "r") as z_list:
        return z_list.read().strip().split(',')

def set_location(code):
    ''' Get the latitude and longitude corrosponding to the zip code.
        
        :param code: the zip code to find weather data about
        :type code: string
    '''
    global obs, zlat, zlon
    try:
        obs = owm.weather_at_zip_code(f'{code}', 'us')
    except APICallTimeoutError:
        time.sleep(5)
        try:
            obs = owm.weather_at_zip_code(f'{code}', 'us')
        except APICallTimeoutError:
            print(f'could not get past the goddamn api call for {code}.')
            return
    location = obs.get_location()
    zlon = location.get_lon()
    zlat = location.get_lat()
    return

In [12]:
va_zips = read_list_from_file('resources/success_zipsVA.csv')
ga_zips = read_list_from_file('resources/success_zipsGA.csv')
sc_zips = read_list_from_file('resources/success_zipsSC.csv')
nc_zips = read_list_from_file('resources/success_zipsNC.csv')
(len(sc_zips) + len(nc_zips)+len(va_zips)+len(ga_zips))/60

62.93333333333333